In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print("GPUs detected:", gpus)

GPUs detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:



!pip install pillow 
!pip install numpy 
!pip install keras 
!pip install matplotlib 
!pip install lxml 
!pip install keras_cv 


  Using cached pillow-11.3.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.0 kB)
Using cached pillow-11.3.0-cp311-cp311-macosx_11_0_arm64.whl (4.7 MB)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
  Using cached matplotlib-3.10.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.0-cp311-cp311-macosx_10_9_universal2.whl.metadata (111 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.4-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.6-cp311-cp311-mac

In [3]:
#if you are using tensorflow metal you cannot use tensorflow hub

import tensorflow as tf
import keras
import numpy as np
import PIL.Image as Image

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization
import matplotlib

import os 
from lxml import etree

/Users/calebkjoseph/Test/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class_ids = [
    "ThumbsDown",
    "ThumbsUp"
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

Function below is able to process the images and annotations and put them in an array

In [ ]:
def image_to_array(path_to_image): 
    """Function loads in a image and outputs them as a numpy array"""
    loaded_image = Image.open(os.path.join(path_to_image)) #opens the image and saves it in loarded image
    Array = np.array(loaded_image)/255 # makes a numpy array of the loaded image and standardizes it's values between 0-1
    return Array

def read_annotation_file(xml_file):
    """
    Reads a Pascal VOC-style XML file and returns bounding boxes as a NumPy array.
    """
    tree = etree.parse(xml_file) #parses xml file
    root = tree.getroot() #gets the top indice of the xml file
    
    boxes = []
    
    for obj in root.findall("object"): #gets the objects in the object subclass
        name = obj.find("name").text #gets what's the name of this object
        bndbox = obj.find("bndbox") #goes into the boundbox subclass
        xmin = int(bndbox.find("xmin").text) #grabs the differenct xmins
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])
    
    return np.array(boxes), name

def get_x_and_y(path_to_images, path_to_annotations):
    """outputs an array of images and an array of annotations given a path to both files"""
    images_array = []
    bboxes_array = []
    names_array = []
    for image in os.listdir(path_to_images): 
        annotation_file_name = os.path.splitext(image)[0] + '.xml' #gets the annotation file name 
        annotation_file_path = os.path.join(path_to_annotations,annotation_file_name) #gets the path 
        BBoxes, Label_Name = read_annotation_file(annotation_file_path) # reads the files
        Image = image_to_array(os.path.join(path_to_images,image)) 
        images_array.append(Image)
        bboxes_array.append(BBoxes)
        names_array.append(Label_Name)
    images_array = np.array(images_array)
    bboxes_array = np.array(bboxes_array) 
    names_array = np.array(names_array)
    names_array = np.reshape(names_array,(198,1))
    return images_array, bboxes_array, names_array



Next extract the training data

In [ ]:
images, bboxes, classes = get_x_and_y("Workspace/GoodOrBad/Images/ProcessedImages/","Workspace/GoodOrBad/Annotations/")

lookup = tf.keras.layers.StringLookup(output_mode="int") #to change the classes from strings to integers
lookup.adapt(classes) #checks how many classes are in classes
classes = lookup(classes) #encodes it
print(classes)
images = tf.constant(images) #creates tensors for each dataset
bboxes = tf.constant(bboxes)
classes = tf.constant(classes)

data = tf.data.Dataset.from_tensor_slices((images, classes, bboxes)) #turns them into a dataset

num_val = int(len(images) * .1) #creates a number of images to partition
val_data = data.take(num_val)
train_data = data.skip(num_val)




(198, 640, 640, 3) (198, 1, 4) (198, 1)
tf.Tensor(
[[2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [1]
 [2]
 [2]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [2]
 [1]
 [1]

2025-09-18 21:05:09.216162: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-09-18 21:05:09.216199: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-18 21:05:09.216206: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-18 21:05:09.216236: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-18 21:05:09.216246: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Next we need to edit the data that way it can be passed into the yolo model

yolo expects datasets to be in this format:

bounding_boxes = {
    # num_boxes may be a Ragged dimension
    'boxes': Tensor(shape=[batch, num_boxes, 4]),
    'classes': Tensor(shape=[batch, num_boxes])
}

In [35]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_dataset(dataset, num_images=4):
    plt.figure(figsize=(12, 12))
    for i, (image, target) in enumerate(dataset.take(num_images)):
        img = image.numpy().astype("float32")

        # target is a dict with "boxes" and "classes"
        boxes = target["boxes"].numpy()
        labels = target["classes"].numpy()

        ax = plt.subplot(2, 2, i+1)
        ax.imshow(img)

        for j, (xmin, ymin, xmax, ymax) in enumerate(boxes):
            rect = patches.Rectangle(
                (xmin, ymin),
                xmax - xmin,
                ymax - ymin,
                linewidth=2,
                edgecolor="red",
                facecolor="none"
            )
            ax.add_patch(rect)

            ax.text(xmin, ymin - 5, str(int(labels[j])),
                    color="yellow", fontsize=10, weight="bold")

        ax.axis("off")
    plt.show()


In [43]:

def load_dataset(images, classes, bbox):
    # Read Image
    
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": tf.cast(bbox, dtype=tf.float32),
    }
    return {"images": tf.cast(images, tf.float32), "bounding_boxes": bounding_boxes}

augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xyxy"),
        keras_cv.layers.RandomShear(
            x_factor=0.2, y_factor=0.2, bounding_box_format="xyxy"
        ),
        keras_cv.layers.JitteredResize(
            target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format="xyxy"
        ),
    ]
)

train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(16)#16 is the batch size times 4 that way it doesn't batch similar images

train_ds = train_ds.padded_batch(4, drop_remainder=True)
#train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)

#same for the valuation dataset
val_ds = val_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(16)
val_ds = val_ds.padded_batch(4, drop_remainder=True)


In [44]:
#make the dictionary in the datasets tuples

def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]




train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
#visualize_dataset(train_ds)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE) #this line makes it so that it loads the next batch as the current is processing


val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

Next we define the model

In [45]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_xs_backbone_coco"  # We will use yolov8 extra small backbone with coco weights but you have other options
)
#below is how you define the model
yolo = keras_cv.models.YOLOV8Detector(
    num_classes= 2,
    bounding_box_format="xyxy",
    backbone=backbone,
    fpn_depth=1,
)

#defining the optimizer
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    global_clipnorm=10.0,
)

yolo.compile(
    optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou"
)


Next we create our callback. Imma be honest idk how this next few lines of code works

In [41]:
class EvaluateCOCOMetricsCallback(keras.callbacks.Callback):
    def __init__(self, data, save_path):
        super().__init__()
        self.data = data
        self.metrics = keras_cv.metrics.BoxCOCOMetrics(
            bounding_box_format="xyxy",
            evaluate_freq=1e9,
        )

        self.save_path = save_path
        self.best_map = -1.0

    def on_epoch_end(self, epoch, logs):
        self.metrics.reset_state()
        for batch in self.data:
            images, y_true = batch[0], batch[1]
            y_pred = self.model.predict(images, verbose=0)
            self.metrics.update_state(y_true, y_pred)

        metrics = self.metrics.result(force=True)
        logs.update(metrics)

        current_map = metrics["MaP"]
        if current_map > self.best_map:
            self.best_map = current_map
            self.model.save(self.save_path)  # Save the model when mAP improves

        return logs

In [47]:
yolo.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    #callbacks=[EvaluateCOCOMetricsCallback(val_ds, "model.h5")],
)




Epoch 1/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - box_loss: 1.6263 - class_loss: 0.6693 - loss: 2.2956 - val_box_loss: 2.6999 - val_class_loss: 1.9908 - val_loss: 4.6907
Epoch 2/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 43s 975ms/step - box_loss: 1.4249 - class_loss: 0.7063 - loss: 2.1312 - val_box_loss: 2.6898 - val_class_loss: 2.1300 - val_loss: 4.8198
Epoch 3/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 32s 722ms/step - box_loss: 1.4225 - class_loss: 0.6217 - loss: 2.0442 - val_box_loss: 1.8611 - val_class_loss: 2.8569 - val_loss: 4.7179
Epoch 4/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 39s 870ms/step - box_loss: 1.3356 - class_loss: 0.6040 - loss: 1.9396 - val_box_loss: 3.3043 - val_class_loss: 2.9250 - val_loss: 6.2292
Epoch 5/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - box_loss: 1.3922 - class_loss: 0.5222 - loss: 1.9144 - val_box_loss: 3.5123 - val_class_loss: 2.5316 - val_loss: 6.0439
Epoch 6/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 23s 512ms/step - box_loss: 1.3212 - class_loss: 0.5556 - loss: 1.8767 - val_box_loss: 3.0318 

In [50]:
yolo.save("yolo1.h5")